In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from time import sleep
from abc import ABC
import pandas as pd

In [19]:
class PageElement(ABC):
    def __init__(self, webdriver, url=''):
        self.webdriver = webdriver
        self.url = url
        
    def localizar(self, locator):
        return self.webdriver.find_element(*locator)
        
    def abrir(self):
        self.webdriver.get(self.url)

class EmissorDeNotas(PageElement):
    refresh_code_button = (By.ID, 'btnAtualizar')
    login_id = (By.ID, 'ctl00_body_tbCpfCnpj')
    password = (By.ID, 'ctl00_body_tbSenha')
    submit = (By.ID, "ctl00_body_btEntrar")          
    emitirnotas = (By.ID, 'ctl00_wpMenuLateral_mnuRotinasn3')
    cpf = (By.ID, 'ctl00_body_tbCPFCNPJTomador')        
    avancar = (By.ID, 'ctl00_body_btAvancar')
    description = (By.ID, 'ctl00_body_tbDiscriminacao')
    value = (By.ID, 'ctl00_body_tbValor')
    emitir = (By.ID, 'ctl00_body_btEmitir')
    voltar = (By.ID, 'ctl00_cphBase_btVoltar') 
    consulta_notas = (By.ID, 'ctl00_wpMenuLateral_mnuRotinasn7')
    nfemitidas = (By.ID, 'ctl00_body_btEmitidas')
    exportarcsv = (By.ID, 'ctl00_cphPopUp_true_btGerar')
    valor_total = (By.XPATH, '//*[@id="ctl00_cphPopUp_tblResumo"]/tbody/tr[3]/td[2]')
        
    def login(self, login, password):        
        self.localizar(self.refresh_code_button).click()
        self.login = self.localizar(self.login_id)
        self.login.send_keys(login)
        self.password = self.localizar(self.password)
        self.password.send_keys(password)
        sleep(20)
        self.localizar(self.submit).click()
        
    def faturar(self): 
        self.localizar(self.emitirnotas).click()
        
    
    def emitir_notas(self, cpf, description, value):
        self.localizar(self.cpf).send_keys(cpf)
        self.localizar(self.avancar).click()
        self.localizar(self.description).send_keys(description)
        if len(str(value)) == 3:
            self.localizar(self.value).send_keys(str(value) + '00')           
        elif len(str(value)) == 5 and str(value)[2] != '.':
            self.localizar(self.value).send_keys(str(value) + '0') 
        else:
            self.localizar(self.value).send_keys(str(value))        
        self.localizar(self.emitir).click()
        self.webdriver.switch_to.alert.accept()
        self.localizar(self.voltar).click()   
    
    def verificar_total(self):
        self.localizar(self.consulta_notas).click()
        self.localizar(self.nfemitidas).click()
        self.webdriver.switch_to.window(self.webdriver.window_handles[-1])        
        return self.localizar(self.valor_total)

In [ ]:
url = 'https://nfe.prefeitura.sp.gov.br/'
CNPJ =  #INCLUIR O CNPJ - APENAS NUMEROS
password = '' # INCLUIR A SENHA 

webdriver = Chrome()

emissor = EmissorDeNotas(webdriver, url)

emissor.abrir()

emissor.login(CNPJ, password)

emissor.faturar()


file_name =  'ExampleSheet.xlsx' #NOME DO ARQUIVO QUE ESTÂO AS NOTAS A SEREM FATURADAS
sheet =  'MesA' #NOME DA ABA A SER FATURADA
df = pd.read_excel(io=file_name, sheet_name=sheet, header=1)
df2 = pd.read_excel(io=file_name, sheet_name=sheet)
total = round(df2.columns[5], 2)

for index, row in df.iterrows():
    emissor.emitir_notas(row['CPF / CNPJ'], row['DESCRIÇÃO'], row['VALOR'] )
    
valortotal = float(emissor.verificar_total().text[3:].replace('.','').replace(',','.'))

assert valortotal == total